<a href="https://colab.research.google.com/github/HyeonsooKim/Stock-Price-Prediction/blob/main/Dacon_Stock_Price_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colab으로 수행
%%capture
import sys

if 'google.colab' in sys.modules:
    # Install packages in Colab
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*
    !pip install -U matplotlib=3.0.3
    !pip install delayed
    !pip install eli5
    !pip install xgboost==1.4.2
    !pip install finance-datareader
    !pip install -U finance-datareader
    !pip install pandas_datareader
    !pip install pdpbox
    !pip install shap
    !pip install forex-python
    !pip install html_table_parser
    !pip install html5lib
    !pip install --upgrade html5lib==1.0b8
    !pip install bs4

In [2]:
#주식 데이터셋 불러오기 위한 모듈
import os
import FinanceDataReader as fdr
from pandas_datareader import data
from tqdm import tqdm

#환율정보 가져오기 위한 모듈
from forex_python.converter import CurrencyRates, CurrencyCodes

#웹크롤링을 위한 모듈
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from html_table_parser import parser_functions as parser

#시계열 데이트타임
from datetime import datetime

#분석
import pandas as pd
import numpy as np

#시각화
import seaborn as sns
import matplotlib.pyplot as plt

#사이킷런 - 데이터 셋 분리(Time이 들어가는 건 시계열)
from sklearn.model_selection import train_test_split, TimeSeriesSplit

#트리 그릴 때 쓰는 모듈
import graphviz
from sklearn.tree import export_graphviz, DecisionTreeRegressor

#EDA용 데이터 프로파일링
from pandas_profiling import ProfileReport

#범주형 데이터 인코더
from category_encoders import OneHotEncoder, OrdinalEncoder, TargetEncoder

#결측치 채우는 모듈 Imputation
from sklearn.impute import SimpleImputer

#값의 범위에 차이가 있을 때 scaler 사용
from sklearn.preprocessing import StandardScaler

#회귀모델
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge

#데이터 분리, 교차검증
from sklearn.model_selection import train_test_split, cross_val_score

from ipywidgets import interact

#파이프라인
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

#앙상블모델 - 랜덤포레스트 분류
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostRegressor

#평가지표
from sklearn.metrics import accuracy_score, plot_confusion_matrix, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, classification_report, roc_auc_score, r2_score, mean_absolute_error

#벡터머신
from sklearn import svm

#특성고를 때
from sklearn.feature_selection import f_regression, SelectKBest

#하이퍼 파라미터 고를 때 교차검증
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold, validation_curve

#랜덤수
from scipy.stats import randint, uniform

#특성 중요도
import eli5
from eli5.sklearn import PermutationImportance

#그래디언트 부스트
import xgboost
from xgboost import XGBClassifier, XGBRegressor

#에이다부스트
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier

#make_pipeline 같은건데 가상의 회귀분석용 데이터 생성
from sklearn.datasets import make_regression

#PDP모듈
from pdpbox import pdp
from pdpbox.pdp import pdp_isolate, pdp_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot

#comma 없애고 실수형으로 바꿔주는 모듈
from locale import atof, setlocale, LC_NUMERIC

#출력창에 생략없이 출력하는 코드
# pd.options.display.max_columns = None
# pd.options.display.max_rows = 999

#오류없애주는 코드
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ignore this warning: https://github.com/dmlc/xgboost/issues/4300
# xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='xgboost')

## Import up sound alert dependencies
from IPython.display import Audio, display

def allDone():
  display(Audio(url='https://freesound.org/data/previews/219/219244_4082826-lq.mp3', autoplay=True))
## Insert whatever audio file you want above
# allDone()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##주식 데이터 불러오기
>Get Stock List

In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/Codestates_Section2(21.08.09~21.09.07)/project'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission_week3.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
371,더네이쳐홀딩스,298540,KOSDAQ
372,코엔텍,029960,KOSDAQ
373,원익홀딩스,030530,KOSDAQ
374,웹케시,053580,KOSDAQ


> Get Data & Modeling

In [5]:
start_date = '20210104'
end_date = '20210903'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 35
HOW MANY "Business_days" : (175, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08


###Baseline 모델의 구성 소개 ( Sample )
* X : (월 ~ 금) * 33주간
* y : (다음주 월 ~ 금) * 33주간
* y_0 : 다음주 월요일
* y_1 : 다음주 화요일
* y_2 : 다음주 수요일
* y_3 : 다음주 목요일
* y_4 : 다음주 금요일
- 이번주 월~금요일의 패턴을 학습해 다음주 월요일 ~ 금요일을 각각 예측하는 모델을 생성

이 과정을 모든 종목(376개)에 적용

In [6]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))
sample.Close = sample.Close.ffill()
sample = pd.pivot_table(data = sample, values = 'Close', columns = 'weekday', index = 'weeknum')
sample.head()

weekday,0,1,2,3,4
weeknum,,,,,
01,83000.0,83900.0,82200.0,82900.0,88800.0
02,91000.0,90600.0,89700.0,89700.0,88000.0
03,85000.0,87000.0,87200.0,88100.0,86800.0
04,89400.0,86700.0,85600.0,83700.0,82000.0
05,83000.0,84400.0,84600.0,82500.0,83500.0


In [7]:
#단순선형회귀모델 생성
model = LinearRegression()

In [8]:
#뒤에서 두 번째까지
x = sample.iloc[0:-2].to_numpy()
x.shape

(33, 5)

In [9]:
y = sample.iloc[1:-1].to_numpy()
y_0 = y[:,0]
y_1 = y[:,1]
y_2 = y[:,2]
y_3 = y[:,3]
y_4 = y[:,4]

y_values = [y_0, y_1, y_2, y_3, y_4]

* 8월 마지막 주 예측

In [10]:
x_public = sample.iloc[-2].to_numpy()

* 예측

In [11]:
predictions = []
for y_value in y_values :
    model.fit(x,y_value)
    prediction = model.predict(np.expand_dims(x_public,0))
    predictions.append(prediction[0])
predictions

[75178.38262350595,
 74035.76768585922,
 73230.0576133645,
 73914.10226623123,
 72972.82124977317]

* 실제 Public 값

In [12]:
sample.iloc[-1].values

array([74600., 76700., 76800., 76000., 76600.])

## 전체 모델링

In [14]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [15]:
model = LinearRegression()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 8월 13일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 8월 20일까지의 데이터를 학습한다.
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 8월 23일부터 8월 27일까지의 데이터를 예측할 것이다.
    
    predictions = []
    for y_value in y_values :
        model.fit(x,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
sample_submission.isna().sum().sum()

100%|██████████| 376/376 [05:51<00:00,  1.07it/s]


0

In [16]:
sample_submission.columns

Index(['Day', '000060', '000080', '000100', '000120', '000150', '000240',
       '000250', '000270', '000660',
       ...
       '330860', '336260', '336370', '347860', '348150', '348210', '352820',
       '357780', '363280', '950130'],
      dtype='object', length=377)

In [22]:
columns = list(sample_submission.columns[1:])

In [18]:
columns = ['Day'] + [str(x).zfill(6) for x in columns]

In [19]:
sample_submission.columns = columns

In [20]:
sample_submission.to_csv('BASELINE_Linear_Week3.csv',index=False)

In [21]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-08-23,24346.980494,32776.312921,61832.552468,170998.590311,89317.699499,17040.932676,51648.621671,82812.013393,104221.519661,661825.293883,50551.159843,228122.597668,34950.583241,60735.402351,19047.494292,2463.031697,25576.706191,5639.545066,374574.173912,56602.290272,22151.188944,33795.759931,10139.854963,8102.736154,30740.984690,151496.251320,48235.882733,69536.091536,48360.245273,263665.144519,301015.939760,82301.833723,115826.706871,35651.188434,33770.164679,25863.204338,140035.587228,210485.832009,100152.337299,...,109471.075147,237508.028001,13506.765306,179062.260228,263214.117515,8344.474389,7833.318361,36005.598908,14161.255147,76170.258289,58033.399372,52684.751330,28409.976035,395740.515674,760117.551290,569722.542935,19835.340989,34603.901001,62046.059738,127526.887321,16306.936718,116621.191935,47369.405201,11317.286961,4548.128503,37853.898269,20642.337327,81906.161290,120541.002588,5753.751143,47791.983291,52098.545952,59475.529141,35349.143294,31815.626604,55124.552696,284561.665158,296211.353861,30256.266207,17677.001745
1,2021-08-24,24694.226217,32646.844048,61674.902092,171527.439516,90299.130276,17308.534120,54544.421995,83597.774290,104411.738505,675858.605112,50519.911999,227931.645963,35502.774025,60776.459142,19304.416088,2542.720546,25584.060907,5588.808119,366287.182147,56753.657672,22267.756931,34849.311177,10104.983657,8088.601124,30431.088206,153243.051495,47606.710087,68699.514251,48171.638344,266851.368662,299680.667691,84217.629414,116228.389991,35966.420763,33649.198234,26220.331699,139762.719469,205476.597076,100060.500998,...,109542.744984,234889.001441,13408.797342,178081.393322,259313.074445,8435.258089,7724.924821,36079.921266,14126.168725,75636.154436,58909.696911,53055.012621,28738.594397,383463.248354,739191.144999,578871.340464,19580.348898,34838.245535,60383.582121,126743.898148,16163.271349,116614.916667,46511.493286,11236.805992,4563.027469,37004.595410,20520.256094,81793.176119,121897.237377,5751.691577,47448.789537,49748.882579,58083.671636,34286.432196,32165.069719,54125.675737,282008.028810,294785.752755,30311.419054,16960.383289
2,2021-08-25,24873.550459,32814.261681,61288.903679,172043.498718,92821.971552,17849.105364,54690.157469,83966.266039,104456.884205,676053.755435,50074.462097,229217.858378,34895.061352,60681.825869,19195.596231,2461.149779,26074.259895,5658.984919,348191.016317,58834.948040,22030.303860,35105.850138,9873.285208,8099.544391,31154.035030,153899.885087,48281.516945,69125.690676,47643.267497,267108.446301,298653.143519,84209.786604,116906.412411,36163.342276,33597.504676,26248.685922,139409.892619,204416.152117,99468.867209,...,109543.207031,237567.311156,13366.800311,175394.187842,253816.815935,8348.496080,7937.509978,36068.077007,14066.301047,74284.101595,55479.870263,52700.251374,28484.143034,380916.785061,739865.787105,586951.608145,19793.353818,34603.535506,60847.559997,127846.783106,16199.609160,115976.594414,47955.453077,11231.220103,4635.456476,36789.092845,20536.688706,77673.251253,117242.875164,5767.271078,48246.298942,50125.502511,58549.689679,34429.084223,31244.456309,55541.116619,279439.565796,294515.225078,30006.243486,16165.477282
3,2021-08-26,24601.643237,33383.218037,61247.878594,172619.481228,92969.416661,17945.703804,57597.362236,84141.214323,101938.876724,674305.517890,50978.870311,228544.202748,34439.818904,60256.850937,19437.031619,2457.189325,25983.177701,5648.890953,326502.8